Let's aim for cleaner code

In [1]:
import geopandas
import pandas as pd
import numpy as np
import os
import re

In [2]:
airports = pd.read_csv("/Users/JO/PhD/hemspy/data/helipad-data/raw-data/helipad-coordinates.csv", sep=";")
airports_gdf = geopandas.GeoDataFrame(airports, geometry=geopandas.points_from_xy(airports.longitude, airports.latitude), crs="EPSG:4326")
airports_gdf = airports_gdf.to_crs("EPSG:32634") #to metric coords
airports_gdf.geometry = airports_gdf.geometry.buffer(distance=airports.radius)
airports_gdf = airports_gdf.to_crs("EPSG:4326") #back to conventional

In [3]:
flights_path = '/Users/JO/PhD/hemspy/data/fr24-data/raw-data-unzipped-rearranged/flights'
flights_files = [f for f in os.listdir(flights_path) if f.endswith('.csv')]

flights_df_list, positions_df_list = [], []
for file in flights_files:
    flights_df_list.append(pd.read_csv(flights_path+'/'+file))

flights_df = pd.concat(flights_df_list, ignore_index=True)

flights_df['flight_id'] = flights_df['flight_id'].astype(int)

positions_path = '/Users/JO/PhD/hemspy/data/fr24-data/raw-data-unzipped-rearranged/positions'
positions_files = [os.path.join(positions_path, file) for file in os.listdir(positions_path) if file.endswith('.csv')]

# List to store DataFrames
positions_df_list = []

regex_pattern = r'_(.*?)\.'
regex = re.compile(regex_pattern)

# Iterate over each file
for file in positions_files:
    # Extract the flight_id from the file name
    flight_id_match = regex.search(os.path.basename(file))
    if flight_id_match:
        flight_id = flight_id_match.group(1)
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file, usecols=['snapshot_id', 'altitude', 'latitude', 'longitude', 'speed'])
        
        # Add a column for flight_id
        df['flight_id'] = flight_id
        
        # Append the DataFrame to the list
        positions_df_list.append(df)

# Combine all DataFrames into a single DataFrame
positions_df = pd.concat(positions_df_list, ignore_index=True)

positions_df['flight_id'] = positions_df['flight_id'].astype(int)

d = pd.merge(positions_df, flights_df, on='flight_id', how='left')

d['UTC'] = pd.to_datetime(d['snapshot_id'], unit='s', utc=True)
d['date'] = d['UTC'].dt.date
d['year'] = d['UTC'].dt.year

# Limit to UAS and OSD to make data smaller
d = d[d['reg'].isin(['SEJSK', 'SEJSI'])]
d = geopandas.GeoDataFrame(d, geometry=geopandas.points_from_xy(d.longitude, d.latitude), crs="EPSG:4326")
d = geopandas.sjoin(d, airports_gdf, how="left", predicate="within")


In [4]:
d.drop(['index_right', 'equip', 'icao', 'is_primary_helipad', 'helipad_location', 'year', 'date', 'is_primary_hospital', 'ambulance_meetup', 'latitude_right', 'reserved', 'flight', 'callsign', 'longitude_right', 'radius', 'real_to', 'schd_from', 'schd_to'], axis=1, inplace=True)

In [5]:
d = d.groupby('aircraft_id').apply(lambda x: x.sort_values('UTC'), include_groups=False)

In [6]:
d.reset_index(drop=True, inplace=True)

In [7]:
d

,snapshot_id,altitude,latitude_left,longitude_left,speed,flight_id,reg,UTC,geometry,hospital_name,zone_name
0,1517660632,725,55.55214,13.37061,73,273769871,SEJSI,2018-02-03 12:23:52+00:00,POINT (13.37061 55.55214),NaN,NaN
1,1517660641,850,55.55391,13.36633,77,273769871,SEJSI,2018-02-03 12:24:01+00:00,POINT (13.36633 55.55391),NaN,NaN
2,1517660650,900,55.55549,13.36066,85,273769871,SEJSI,2018-02-03 12:24:10+00:00,POINT (13.36066 55.55549),NaN,NaN
3,1517660656,925,55.55644,13.35713,89,273769871,SEJSI,2018-02-03 12:24:16+00:00,POINT (13.35713 55.55644),NaN,NaN
4,1517660662,1025,55.55759,13.35322,85,273769871,SEJSI,2018-02-03 12:24:22+00:00,POINT (13.35322 55.55759),NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2764173,1715776157,0,59.89304,17.60881,26,893015657,SEJSK,2024-05-15 12:29:17+00:00,POINT (17.60881 59.89304),Akademiska sjukhuset,Ärna flygplats
2764174,1715776160,0,59.89290,17.60824,24,893015657,SEJSK,2024-05-15 12:29:20+00:00,POINT (17.60824 59.89290),Akademiska sjukhuset,Ärna flygplats
2764175,1715776164,0,59.89275,17.60770,18,893015657,SEJSK,2024-05-15 12:29:24+00:00,POINT (17.60770 59.89275),Akademiska sjukhuset,Ärna flygplats
2764176,1715776169,0,59.89262,17.60739,9,893015657,SEJSK,2024-05-15 12:29:29+00:00,POINT (17.60739 59.89262),Akademiska sjukhuset,Ärna flygplats


In [8]:
# Identify transitions into and out of helipad zones
d['in_helipad_zone'] = ~d['zone_name'].isna()
d['zone_change'] = d['in_helipad_zone'].ne(d['in_helipad_zone'].shift())

In [9]:
# Extract the entry and exit times
d['entry'] = (d['zone_change'] == True) & (d['in_helipad_zone'] == True)
d['exit'] = (d['zone_change'] == True) & (d['in_helipad_zone'] == False)
d['UTC_str'] = d['UTC'].astype(str)

In [10]:
entries_and_exits = d[d['entry'] | d['exit']]

In [203]:
check_flight = np.random.choice(d.flight_id)
entries_and_exits.query(f"flight_id == {check_flight}")

,snapshot_id,altitude,latitude_left,longitude_left,speed,flight_id,reg,UTC,geometry,hospital_name,zone_name,in_helipad_zone,zone_change,entry,exit,UTC_str
2033162,1653737611,900,59.90188,17.62482,99,738656585,SEJSK,2022-05-28 11:33:31+00:00,POINT (17.62482 59.90188),Akademiska sjukhuset,Ärna flygplats,True,True,True,False,2022-05-28 11:33:31+00:00


In [204]:
m=airports_gdf.explore()
d[['geometry', 'flight_id', 'UTC_str']].query(f"flight_id == {check_flight}").explore(m=m)

In [13]:
d[['geometry', 'flight_id', 'UTC_str']].query(f"flight_id == {check_flight}").iloc[-2:].explore()

the stuff of nightmares

### Testing hidden markov models

In [14]:
# calc ROC
def ROC(x):
    # Calculate distance
    point1 = x.geometry.to_crs("EPSG:32634")
    point2 = x.shift().geometry.to_crs("EPSG:32634")
    distance = point1.distance(point2)
    # Calculate change in altitude
    alt1 = x.altitude
    alt2 = x.shift().altitude
    alt_change = alt1 - alt2
    # Calculate rate of climb
    return alt_change / distance

def DELTA_SPEED(x):
     # Calculate distance
    point1 = x.geometry.to_crs("EPSG:32634")
    point2 = x.shift().geometry.to_crs("EPSG:32634")
    time = x.snapshot_id - x.shift().snapshot_id

    speed1 = x.speed
    speed2 = x.shift().speed
    return (speed1 - speed2) / time

def ASC(x):
    alt1 = x.altitude
    alt2 = x.shift().altitude
    return alt1 > alt2

In [15]:
distances = d.groupby('flight_id').apply(lambda x: ROC(x), include_groups=False)
d_speed = d.groupby('flight_id').apply(lambda x: DELTA_SPEED(x), include_groups=False)
ascent = d.groupby('flight_id').apply(lambda x: ASC(x), include_groups=False)
ROC = distances.reset_index(drop=True)
D_SPEED = d_speed.reset_index(drop=True)
ASC = ascent.reset_index(drop=True)

In [16]:
d['ROC'] = ROC
d['dSpeed'] = D_SPEED
d['ASC'] = ASC

In [25]:
d['ROC'] = np.where(d.ROC ==  np.inf, 0, d.ROC)
d['ROC'] = np.where(d.ROC ==  -np.inf, 0, d.ROC)
d['ROC'] = d['ROC'].fillna(0)

In [26]:
d['dSpeed'] = d['dSpeed'].fillna(0)
d['ASC'] = d['ASC'].fillna(0)

In [27]:
d

,snapshot_id,altitude,latitude_left,longitude_left,speed,flight_id,reg,UTC,geometry,hospital_name,zone_name,in_helipad_zone,zone_change,entry,exit,UTC_str,ROC,dSpeed,ASC
0,1517660632,725,55.55214,13.37061,73,273769871,SEJSI,2018-02-03 12:23:52+00:00,POINT (13.37061 55.55214),NaN,NaN,False,True,False,True,2018-02-03 12:23:52+00:00,0.000000,0.000000,False
1,1517660641,850,55.55391,13.36633,77,273769871,SEJSI,2018-02-03 12:24:01+00:00,POINT (13.36633 55.55391),NaN,NaN,False,False,False,False,2018-02-03 12:24:01+00:00,0.372942,0.444444,True
2,1517660650,900,55.55549,13.36066,85,273769871,SEJSI,2018-02-03 12:24:10+00:00,POINT (13.36066 55.55549),NaN,NaN,False,False,False,False,2018-02-03 12:24:10+00:00,0.125094,0.888889,True
3,1517660656,925,55.55644,13.35713,89,273769871,SEJSI,2018-02-03 12:24:16+00:00,POINT (13.35713 55.55644),NaN,NaN,False,False,False,False,2018-02-03 12:24:16+00:00,0.101132,0.666667,True
4,1517660662,1025,55.55759,13.35322,85,273769871,SEJSI,2018-02-03 12:24:22+00:00,POINT (13.35322 55.55759),NaN,NaN,False,False,False,False,2018-02-03 12:24:22+00:00,0.358860,-0.666667,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2764173,1715776157,0,59.89304,17.60881,26,893015657,SEJSK,2024-05-15 12:29:17+00:00,POINT (17.60881 59.89304),Akademiska sjukhuset,Ärna flygplats,True,False,False,False,2024-05-15 12:29:17+00:00,0.000000,-0.333333,False
2764174,1715776160,0,59.89290,17.60824,24,893015657,SEJSK,2024-05-15 12:29:20+00:00,POINT (17.60824 59.89290),Akademiska sjukhuset,Ärna flygplats,True,False,False,False,2024-05-15 12:29:20+00:00,0.000000,-0.666667,False
2764175,1715776164,0,59.89275,17.60770,18,893015657,SEJSK,2024-05-15 12:29:24+00:00,POINT (17.60770 59.89275),Akademiska sjukhuset,Ärna flygplats,True,False,False,False,2024-05-15 12:29:24+00:00,0.000000,-1.500000,False
2764176,1715776169,0,59.89262,17.60739,9,893015657,SEJSK,2024-05-15 12:29:29+00:00,POINT (17.60739 59.89262),Akademiska sjukhuset,Ärna flygplats,True,False,False,False,2024-05-15 12:29:29+00:00,0.000000,-1.800000,False


In [78]:
features = d[['speed', 'altitude', 'ROC']]

In [79]:
from sklearn.preprocessing import StandardScaler

In [80]:
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

In [81]:
features_scaled

array([[-1.22338766, -0.79335971,  0.00807967],
       [-1.10841619, -0.69231603,  0.17687697],
       [-0.87847324, -0.65189856,  0.06469865],
       ...,
       [-2.80424543, -1.37941303,  0.00807967],
       [-3.06293125, -1.37941303,  0.00807967],
       [-3.32161707, -1.37941303,  0.00807967]])

In [82]:
x = pd.DataFrame(features_scaled)
x['flight_id'] = d['flight_id']
x_list = []
x.groupby('flight_id').apply(lambda x: x_list.append(x.values), include_groups=False)

""


In [83]:
lengths = []
x.groupby('flight_id').apply(lambda x: lengths.append(len(x)), include_groups=False)

""


In [84]:
from hmmlearn import hmm

In [148]:
n_states = 3
model = hmm.GaussianHMM(n_components=n_states, n_iter=50, covariance_type='spherical')

# Fit the model
model.fit(features_scaled, lengths)
states = model.predict(features_scaled)

d['states'] = states

In [149]:
model.monitor_.converged

True

In [150]:
d.states.value_counts()

states
0    2273250
1     470425
2      20503
Name: count, dtype: int64

In [151]:
d.groupby('states').agg({
    'speed': ['mean', 'std'],
    'altitude': ['mean', 'std'],
    'ROC': ['mean', 'std'],
    'ASC': ['mean', 'std'],
    'dSpeed': ['mean', 'std'],
})


speed                altitude                    ROC             \
              mean        std         mean          std      mean        std   
states                                                                         
0       128.484039  16.457861  1884.370414  1143.308199 -0.012367   0.617231   
1        52.111773  30.170302   646.535514   512.294714 -0.017559   0.374058   
2       138.816563  29.079086  6299.455007  2234.317981 -0.632685  24.744917   

             ASC              dSpeed            
            mean       std      mean       std  
states                                          
0       0.183729  0.387263 -0.023794  0.810455  
1       0.182314  0.386103 -0.031099  0.757219  
2       0.219139  0.413673  0.005359  0.726200

In [164]:
check_flight = np.random.choice(d.flight_id)
#check_flight= 686697156 #flyover västerås
color_map = {
    0: 'blue',
    1: 'red',
    2: 'green',
    3: 'orange',
    4: 'maroon',
    5: 'black'
}
m=airports_gdf.explore()

d[['geometry', 'flight_id', 'altitude', 'speed', 'ROC', 'dSpeed', 'states']].query(f"flight_id == {check_flight}").explore(m=m,color=d['states'].map(color_map))

Conclusion: I can train a HMM to infer two states: flying and landing/takeoff. Problem is that it is mostly only using altitude/speed as features. If the take off is captured "late" it looks as if it is flying at level height. Also, it is very black boxy.

Maybe the best is to do large bounding boxes and work with entries/exits.